In [ ]:
import simpy
import pandas as pd 
import numpy as np 

In [ ]:
df = np.random.rand(1000,4)
df = pd.DataFrame(df, dtype=np.float16)
df

In [ ]:
%%timeit

df.loc[df[0] > 0.5].sort_values(0)

In [ ]:
%%timeit

df.sort_values(0).loc[df[0] > 0.5]

In [ ]:
import random

dcl = []

for a in range(1000):
    dcl.append(
        {
           0: np.random.rand(),
           1: np.random.rand(),
           2: np.random.rand(),
           3: np.random.rand(), 
        }
    )     


In [ ]:
%%timeit
list(filter(
    lambda x: x[0] > 0.5, dcl
))

In [ ]:
%%timeit
list(filter(
    lambda x: x[0] > 0.5, sorted(
        dcl, key=lambda y: y[0]
    )
))


In [ ]:
%%timeit
list(
    sorted(
        filter(
            lambda x: x[0]>5, dcl
        ), key=lambda y: y[0]
    )
)


In [ ]:
from collections import namedtuple

Machine = namedtuple('Machine', 'size, duration')
m1 = Machine(1, 2)  # Small and slow
m2 = Machine(2, 1)  # Big and fast

env = simpy.Environment()
machine_shop = simpy.FilterStore(env, capacity=2)
machine_shop.items = [m1, m2]  # Pre-populate the machine shop

def user(name, env, ms, size):
    machine = yield ms.get(lambda machine: machine.size == size)
    print(name, 'got', machine, 'at', env.now)
    yield env.timeout(machine.duration)
    # yield ms.put(machine)
    print(name, 'released', machine, 'at', env.now)


users = [env.process(user(i, env, machine_shop, 2))
         for i in range(3)]
env.run()

In [ ]:
a = machine_shop.items
a[0] = Machine(20,2)
machine_shop.items = a
machine_shop.items

In [ ]:
type(a)